# **Converting to TFLite :**

In [1]:
# Importing dependancies
from tensorflow import lite
import numpy as np

2021-10-15 03:59:34.888132: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/conda/lib
2021-10-15 03:59:34.888240: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
# Initialising converters
encoder_tflite_converter = lite.TFLiteConverter.from_saved_model("../input/neural-machine-translation-english-to-italian/encoder")
decoder_tflite_converter = lite.TFLiteConverter.from_saved_model("../input/neural-machine-translation-english-to-italian/decoder")

2021-10-15 03:59:39.323925: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-10-15 03:59:39.326403: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/conda/lib
2021-10-15 03:59:39.326445: W tensorflow/stream_executor/cuda/cuda_driver.cc:326] failed call to cuInit: UNKNOWN ERROR (303)
2021-10-15 03:59:39.326471: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (91b52915cfa0): /proc/driver/nvidia/version does not exist
2021-10-15 03:59:39.326727: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operation

In [3]:
# Defining quantisation schema
encoder_tflite_converter.optimizations = [lite.Optimize.DEFAULT]
decoder_tflite_converter.optimizations = [lite.Optimize.DEFAULT]

In [4]:
# Converting to TFLite
encoder_tflite_content = encoder_tflite_converter.convert()
decoder_tflite_content = decoder_tflite_converter.convert()

2021-10-15 03:59:54.306700: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:316] Ignored output_format.
2021-10-15 03:59:54.306789: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:319] Ignored drop_control_dependency.
2021-10-15 03:59:54.306800: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:325] Ignored change_concat_input_ranges.
2021-10-15 03:59:54.307977: I tensorflow/cc/saved_model/reader.cc:32] Reading SavedModel from: ../input/neural-machine-translation-english-to-italian/encoder
2021-10-15 03:59:54.328374: I tensorflow/cc/saved_model/reader.cc:55] Reading meta graph with tags { serve }
2021-10-15 03:59:54.328437: I tensorflow/cc/saved_model/reader.cc:93] Reading SavedModel debug info (if present) from: ../input/neural-machine-translation-english-to-italian/encoder
2021-10-15 03:59:54.328551: I tensorflow/compiler/jit/xla_gpu_device.cc:99] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-10-15 03:59:54.

In [5]:
# Saving quantised models
with open("./encoder.tflite", 'wb') as f:
    f.write(encoder_tflite_content)

with open("./decoder.tflite", 'wb') as f:
    f.write(decoder_tflite_content)

e it has no allocated buffer.


# **Inference from quantised model :**

In [6]:
# Importing Dependancies
import pickle
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

In [7]:
# Loading tokenizers
eng_tok = pickle.load(open("../input/neural-machine-translation-english-to-italian/English Tokenizer.pkl", 'rb'))
it_tok = pickle.load(open("../input/neural-machine-translation-english-to-italian/Italian Tokenizer.pkl", 'rb'))

In [8]:
eng_seq_len = 20    # First dimension of encoder Input shape
eng_vocab_size = len(eng_tok.word_index)+1    # Second dimension of encoder Input shape

it_seq_len = 20     # First dimension of decoder Input shape
it_vocab_size = len(it_tok.word_index)+1     # Second dimension of decoder Input shape

In [9]:
def sent_to_seq(sequences, tokenizer, vocab_size=None, reverse=False, onehot=False):
    
    """ Converts text data into sequences supported by model input layers.
    
    Args:
        sequences (list): List of text data.
        tokenizer (tf.keras.preprocessing.text.Tokenizer): Tensorflow tokenizer object.
        vocab_size (int): Number of words in the whole vocabulary.
        reverse (bool): Reverses the padded sequence if set True. Defaults False.
                        (Eg: if set True, [1 2 3 0 0] becomes [0 0 3 2 1])
        onehot (bool): Creates onehot representation of the padded sequence if set True.
                       Defaults False.
                       
    Returns:
        preprocessed_seq (list): List of preprocessed sequences.
        
    """
    
    # Tokenizing
    seq = tokenizer.texts_to_sequences(sequences)
    
    # Padding
    preprocessed_seq = pad_sequences(seq, padding='post', truncating='post', maxlen=20)
    
    # Reversing
    if reverse:
        preprocessed_seq = preprocessed_seq[:, ::-1]
    
    # Onehot encoding
    if onehot:
        preprocessed_seq = to_categorical(preprocessed_seq, num_classes=vocab_size) 
    
    return preprocessed_seq

            

def word_to_onehot(tokenizer, word, vocab_size):
    
    """ Converts a single word into onehot representation.
    
    Args:
        tokenizer (tf.keras.preprocessing.text.Tokenizer): Tensorflow tokenizer object.
        word (str): Word to be tokenized and onehot encoded.
        vocab_size (int): Number of words in the whole vocabulary.
    
    Returns:
        de_onhot (list): Onehot representation of given word.
        
    """
    
    de_seq = tokenizer.texts_to_sequences([[word]])
    de_onehot = to_categorical(de_seq, num_classes=vocab_size).reshape(1, 1, vocab_size)  
    
    return de_onehot



In [10]:
# Loading TFLite models
enc_interpreter = lite.Interpreter(model_path="./encoder.tflite")
dec_interpreter = lite.Interpreter(model_path="./decoder.tflite")

# Allocates tensors
enc_interpreter.allocate_tensors()
dec_interpreter.allocate_tensors()

In [11]:
# Input layer details
en_input_details = enc_interpreter.get_input_details()
de_input_details = dec_interpreter.get_input_details()

# Output layer details
en_output_details = enc_interpreter.get_output_details()
de_output_details = dec_interpreter.get_output_details()

print(enc_interpreter.get_input_details())

[{'name': 'serving_default_input_3:0', 'index': 0, 'shape': array([    1,    20, 13893], dtype=int32), 'shape_signature': array([   -1,    20, 13893], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}]


In [12]:
print(de_input_details[1])

{'name': 'serving_default_input_4:0', 'index': 1, 'shape': array([    1,     1, 28445], dtype=int32), 'shape_signature': array([   -1,     1, 28445], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}


In [13]:
print(de_output_details)

[{'name': 'StatefulPartitionedCall:1', 'index': 25, 'shape': array([   1, 2048], dtype=int32), 'shape_signature': array([  -1, 2048], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}, {'name': 'StatefulPartitionedCall:0', 'index': 30, 'shape': array([    1, 28445], dtype=int32), 'shape_signature': array([   -1, 28445], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}]


In [14]:
def translate(eng_sentence):
    
    """ Returns Italian translation of given english sentence.
    
    Args:
        eng_sentence (str): English text to be translated.
        
    Returns:
        it_sent (str): Italian translated text.
        
    """
    
    en_seq = sent_to_seq([eng_sentence], 
                         tokenizer=eng_tok, 
                         reverse=True, 
                         onehot=True, 
                         vocab_size=eng_vocab_size)
    
    enc_interpreter.set_tensor(en_input_details[0]['index'], en_seq)
    enc_interpreter.invoke()
    
    en_st = enc_interpreter.get_tensor(en_output_details[0]['index'])
    de_seq = word_to_onehot(it_tok, "sos", it_vocab_size)

    it_sent = ""
    for i in range(it_seq_len):
        dec_interpreter.set_tensor(de_input_details[0]['index'], en_st)
        dec_interpreter.set_tensor(de_input_details[1]['index'], de_seq)
        dec_interpreter.invoke()
        en_st = dec_interpreter.get_tensor(de_output_details[0]['index'])
        de_prob = dec_interpreter.get_tensor(de_output_details[1]['index'])
        index = np.argmax(de_prob[0, :], axis=-1)
        de_w = it_tok.index_word[index]
        de_seq = word_to_onehot(it_tok, de_w, it_vocab_size) 
        if de_w == 'eos': break
        it_sent += de_w + ' '
        
    return it_sent

In [15]:
# Making translations
print(translate("Can I talk to you for a minute?"))

posso parlarvi per un minuto 
